# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
from scipy.spatial.distance import pdist, squareform

In [4]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [122]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [16]:
df = data[['CustomerID','ProductName','Quantity']].groupby(['CustomerID','ProductName','Quantity']).sum().reset_index()

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [19]:
dfPivot = pd.pivot_table(df, values='Quantity', index='ProductName',columns='CustomerID',fill_value=0)

## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [26]:
distances = pdist(dfPivot.T, 'euclidean')
distances_matrix = squareform(distances)
dfDistances = pd.DataFrame(1/(1 + distances_matrix), index=dfPivot.columns, columns=dfPivot.columns)

In [61]:
dfDistances.columns

Int64Index([   33,   200,   264,   356,   412,   464,   477,   639,   649,
              669,
            ...
            97697, 97753, 97769, 97793, 97900, 97928, 98069, 98159, 98185,
            98200],
           dtype='int64', name='CustomerID', length=1000)

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [64]:
dfDistances.reset_index()[['CustomerID',33]].sort_values(by=33,ascending=False).head(6)
#Los 6 primeros para el cliente 33, obviamente el match consigo mismo es 1, la primera columna que pone
#Customer id enrealidad son los índices de fila, los customerId verdaderos son los de enmedio

CustomerID,CustomerID,33
0,33,1.000000
51,3909,0.095358
46,3531,0.093953
2,264,0.093953
29,2503,0.093498
43,3305,0.093051


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [72]:
dfSimilar = df[df.CustomerID.isin([3909,3531,264,2503,3305])].reset_index().drop(columns='index')
dfSimilar.head()

,CustomerID,ProductName,Quantity
0,264,Apricots - Halves,1
1,264,Apricots Fresh,1
2,264,Bacardi Breezer - Tropical,1
3,264,Bagel - Plain,1
4,264,Banana - Leaves,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [85]:
dfRank = dfSimilar[['ProductName','Quantity']].groupby('ProductName').sum().sort_values(by='Quantity',ascending=False).reset_index()
dfRank.head()

,ProductName,Quantity
0,Pecan Raisin - Tarts,3
1,Quiche Assorted,3
2,Potatoes - Idaho 100 Count,2
3,Chef Hat 20cm,2
4,Coffee Decaf Colombian,2


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [93]:
df33 = dfPivot.reset_index()[['ProductName',33]]

dfRank.merge(df33[df33[33]==0], left_on='ProductName', right_on='ProductName').head(5)

,ProductName,Quantity,33
0,Pecan Raisin - Tarts,3,0
1,Chef Hat 20cm,2,0
2,Coffee Decaf Colombian,2,0
3,"Beets - Candy Cane, Organic",2,0
4,Coffee - Dark Roast,2,0


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [111]:
def reco(idC):
    listSimilar = list(dfDistances.reset_index()[['CustomerID',idC]].sort_values(by=idC,ascending=False)['CustomerID'])[1:6]
    dfSimilar = df[df.CustomerID.isin(listSimilar)].reset_index().drop(columns='index')
    dfid = dfPivot.reset_index()[['ProductName',idC]]
    listaReco = list(dfRank.merge(dfid[dfid[idC]==0], left_on='ProductName', right_on='ProductName').head(5)['ProductName'])
    return {'CustomerId':idC,
            'Reco1':listaReco[0],
           'Reco2':listaReco[1],
           'Reco3':listaReco[2],
           'Reco4':listaReco[3],
           'Reco5':listaReco[4]}
    
    

In [118]:
listRecos=[]
for ele in data['CustomerID'].unique():
    listRecos.append(reco(ele))

In [120]:
len(listRecos)

1000

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [124]:
pd.DataFrame(listRecos).head()

,CustomerId,Reco1,Reco2,Reco3,Reco4,Reco5
0,61288,Pecan Raisin - Tarts,Quiche Assorted,Potatoes - Idaho 100 Count,Chef Hat 20cm,Coffee Decaf Colombian
1,77352,Potatoes - Idaho 100 Count,Chef Hat 20cm,Coffee Decaf Colombian,Pork - Hock And Feet Attached,Black Currants
2,40094,Pecan Raisin - Tarts,Quiche Assorted,Chef Hat 20cm,Coffee Decaf Colombian,"Beets - Candy Cane, Organic"
3,23548,Pecan Raisin - Tarts,Quiche Assorted,Potatoes - Idaho 100 Count,Chef Hat 20cm,Coffee Decaf Colombian
4,78981,Quiche Assorted,Potatoes - Idaho 100 Count,Chef Hat 20cm,Coffee Decaf Colombian,"Beets - Candy Cane, Organic"


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.

In [125]:
distances = pdist(dfPivot.T, 'cosine')
distances_matrix = squareform(distances)
dfDistances = pd.DataFrame(1/(1 + distances_matrix), index=dfPivot.columns, columns=dfPivot.columns)
listRecos=[]
for ele in data['CustomerID'].unique():
    listRecos.append(reco(ele))

In [126]:
pd.DataFrame(listRecos).head() #El resultado es el mismo porque ambas métricas son buenas para medir por cantidades

,CustomerId,Reco1,Reco2,Reco3,Reco4,Reco5
0,61288,Pecan Raisin - Tarts,Quiche Assorted,Potatoes - Idaho 100 Count,Chef Hat 20cm,Coffee Decaf Colombian
1,77352,Potatoes - Idaho 100 Count,Chef Hat 20cm,Coffee Decaf Colombian,Pork - Hock And Feet Attached,Black Currants
2,40094,Pecan Raisin - Tarts,Quiche Assorted,Chef Hat 20cm,Coffee Decaf Colombian,"Beets - Candy Cane, Organic"
3,23548,Pecan Raisin - Tarts,Quiche Assorted,Potatoes - Idaho 100 Count,Chef Hat 20cm,Coffee Decaf Colombian
4,78981,Quiche Assorted,Potatoes - Idaho 100 Count,Chef Hat 20cm,Coffee Decaf Colombian,"Beets - Candy Cane, Organic"
